In [8]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [9]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2022
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2022',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 495,
    'OBJECTID': '495',
    'Year': '2022',
    'State': 'CA',
    'Agency': 'USF',
    'Unit ID': 'SRF',
    'Fire Name': 'AMMON',
    'Local Incident Number': '00000635',
    'Alarm Date': '8/5/2022 12:00:00 AM',
    'Containment Date': '9/6/2022 12:00:00 AM',
    'Cause': '14',
    'Collection Method': '3',
    'Management Objective': '1',
    'GIS Calculated Acres': '11464.9',
    'Comments': None,
    'Complex Name': '2022 SRF Lightning Complex',
    'IRWIN ID': '{C403F1AF-8762-41CE-A75E-5A4508B122E2}',
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'Shape__Area': '81126625.4179688',
    'Shape__Length': '64551.6567668991',
    'rank': 0.0706241},
  

In [10]:
df = pd.DataFrame(response['result']['records'])
df_2022 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2022.columns if col not in columns_to_keep]
df_2022 = df_2022.drop(columns=columns_to_drop)
df_2022.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2022,8/5/2022 12:00:00 AM,9/6/2022 12:00:00 AM,14,11464.9
1,2022,8/5/2022 12:00:00 AM,9/9/2022 12:00:00 AM,14,30132.85
2,2022,5/24/2022 12:00:00 AM,5/24/2022 12:00:00 AM,14,29.84678
3,2022,6/13/2022 12:00:00 AM,6/19/2022 12:00:00 AM,14,589.8671
4,2022,6/3/2022 12:00:00 AM,6/3/2022 12:00:00 AM,7,28.68488


In [11]:
df_2022['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2022['Alarm Date']]
df_2022['Containment Date'] = [str(item) for item in df_2022['Containment Date']]
df_2022['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2022['Containment Date']]
df_2022.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2022,8/5/2022,9/6/2022,14,11464.9
1,2022,8/5/2022,9/9/2022,14,30132.85
2,2022,5/24/2022,5/24/2022,14,29.84678
3,2022,6/13/2022,6/19/2022,14,589.8671
4,2022,6/3/2022,6/3/2022,7,28.68488


In [12]:
df_2022 = df_2022.rename(columns={'Cause': 'Cause #'})
df_2022['Cause'] = ''
df_2022['Cause #'] = pd.to_numeric(df_2022['Cause #'], errors='coerce')
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2022['Cause'] = df_2022['Cause #'].apply(map_cause_to_meaning)
df_2022.head()


,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2022,8/5/2022,9/6/2022,14,11464.9,Unknown / Unidentified
1,2022,8/5/2022,9/9/2022,14,30132.85,Unknown / Unidentified
2,2022,5/24/2022,5/24/2022,14,29.84678,Unknown / Unidentified
3,2022,6/13/2022,6/19/2022,14,589.8671,Unknown / Unidentified
4,2022,6/3/2022,6/3/2022,7,28.68488,Arson


In [14]:
output_path = os.path.join("outputs", "timeline_dates.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])

NameError: name 'earliest_fire' is not defined